# Ingeligencja obliczeniowa

**Projekt 5: Przestrzenie Ciągłe**   
Olgierd Piofczyk, Kaja Dzielnicka

---

## Środowisko: Pendulum-v1

W ramach projektu zastosowaliśmy środowisko `Pendulum-v1` z biblioteki `gym`. Jest to klasyczny problemem sterowania, w którym zadaniem agenta jest balansowanie wahadła w pionie, startując z dowolnej pozycji. Agent otrzymuje negatywne nagrody za odchylenia od pionu oraz za dużą prędkość kątową.

## Algorytm: SAC

SAC jest algorytmem wzmacniającego uczenia, który łączy metody aktora-krytyka z maksymalizacją entropii. Celem jest nie tylko maksymalizacja nagrody, ale także eksploracja różnych działań.

## Implementacja

### Instalacja środowiska

In [ ]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import SAC
import matplotlib.pyplot as plt
import os

### Zdefiniowanie folderu do zapisywania wyników

In [ ]:
logs_dir = 'logs'
learning_algorithm = 'SAC'

if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

if not os.path.exists(learning_algorithm):
    os.makedirs(learning_algorithm)

### Metoda uczenia

In [ ]:
def train(env, max_iters=10, timesteps=50000):

    model = SAC('MlpPolicy', env, verbose=1, tensorboard_log=logs_dir)
    name = f'{learning_algorithm}/'

    for iters in range(max_iters):
        model.learn(total_timesteps=timesteps, reset_num_timesteps=False)
        model.save(f'{name}{timesteps*iters}')

### Metoda testowania

In [ ]:
def test(env, path_to_model):
    model = SAC.load(path_to_model)

    obs = env.reset()[0]
    done = False
    while True:
        action, _states = model.predict(obs)
        obs, _, done, _, _ = env.step(action)
        
        if done:
            break

### Definicja środowiska i trening agenta

In [ ]:
env = gym.make('Pendulum-v1', render_mode=None)
env_test = gym.make('Pendulum-v1', render_mode='human')

train(env, 'SAC')

In [ ]:
test(env_test, 'SAC', '')